In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



**Imports**

In [ ]:
import numpy as np
import json

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
# from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler
%matplotlib inline

Check if gpu is enabled

In [ ]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

Loading Dataset

In [ ]:
dataset_path = "/content/drive/MyDrive/mashfi/GTZAN_AdvarsarialAttack/Data/images_original/"

In [ ]:

original_dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform = transforms.Compose([transforms.Resize([32, 32]),transforms.ToTensor()]))

In [ ]:
#dataset train test split
n = len(original_dataset)  # total number of examples
n_test = int(0.15 * n)  # take ~33% for test
test_set = torch.utils.data.Subset(original_dataset, range(n_test))  # take first 10%
train_set = torch.utils.data.Subset(original_dataset, range(n_test, n))  # take the rest   

In [ ]:
original_dataset.class_to_idx

{'blues': 0,
 'classical': 1,
 'country': 2,
 'disco': 3,
 'hiphop': 4,
 'jazz': 5,
 'metal': 6,
 'pop': 7,
 'reggae': 8,
 'rock': 9}

**Prepares the Dataset**

In [ ]:
# MNIST Train dataset and dataloader declaration
trainloader = torch.utils.data.DataLoader(train_set,batch_size=100, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_set,batch_size=100, shuffle=True, num_workers=2)



Map Images with Labels from test/train loader

In [ ]:
normal_iter = iter(trainloader)
images, labels = normal_iter.next()

True Image & True Label


In [ ]:
print(type(images))
print(f"image size {images.shape}")
print(f"labels size {labels.shape}")

<class 'torch.Tensor'>
image size torch.Size([100, 3, 32, 32])
labels size torch.Size([100])


VGG16 Network



In [ ]:
class VGG16(nn.Module):
  def __init__(self):
    super(VGG16,self).__init__()

    self.block1 = nn.Sequential(
                  nn.Conv2d(in_channels = 3,out_channels = 64,kernel_size = 3,padding = 1),
                  nn.BatchNorm2d(64),
                  nn.ReLU(),
                  nn.Conv2d(in_channels = 64,out_channels = 64,kernel_size = 3, padding =1),
                  nn.BatchNorm2d(64),
                  nn.ReLU(),
                  nn.MaxPool2d(kernel_size=2, stride=2),
                  nn.Dropout2d(0.3))

    self.block2 = nn.Sequential(
                  nn.Conv2d(in_channels = 64,out_channels = 128,kernel_size = 3,padding = 1),
                  nn.BatchNorm2d(128),
                  nn.ReLU(),
                  nn.Conv2d(in_channels = 128,out_channels = 128,kernel_size = 3, padding =1),
                  nn.BatchNorm2d(128),
                  nn.ReLU(),
                  nn.MaxPool2d(kernel_size=2, stride=2),
                  nn.Dropout2d(0.4))

    self.block3 = nn.Sequential(
                  nn.Conv2d(in_channels = 128,out_channels = 256,kernel_size = 3,padding = 1),
                  nn.BatchNorm2d(256),
                  nn.ReLU(),
                  nn.Conv2d(in_channels = 256,out_channels = 256,kernel_size = 3,padding = 1),
                  nn.BatchNorm2d(256),
                  nn.ReLU(),
                  nn.Conv2d(in_channels = 256,out_channels = 256,kernel_size = 3, padding =1),
                  nn.BatchNorm2d(256),
                  nn.ReLU(),
                  nn.MaxPool2d(kernel_size=2, stride=2),
                  nn.Dropout2d(0.4))

    self.block4 = nn.Sequential(
                  nn.Conv2d(in_channels = 256,out_channels = 512,kernel_size = 3,padding = 1),
                  nn.BatchNorm2d(512),
                  nn.ReLU(),
                  nn.Conv2d(in_channels = 512,out_channels = 512,kernel_size = 3,padding = 1),
                  nn.BatchNorm2d(512),
                  nn.ReLU(),
                  nn.Conv2d(in_channels = 512,out_channels = 512,kernel_size = 3, padding =1),
                  nn.BatchNorm2d(512),
                  nn.ReLU(),
                  nn.MaxPool2d(kernel_size=2, stride=2) ,
                  nn.Dropout2d(0.4))

    self.block5 = nn.Sequential(
                  nn.Conv2d(in_channels = 512,out_channels = 512,kernel_size = 3,padding = 1),
                  nn.BatchNorm2d(512),
                  nn.ReLU(),
                  nn.Conv2d(in_channels = 512,out_channels = 512,kernel_size = 3,padding = 1),
                  nn.BatchNorm2d(512),
                  nn.ReLU(),
                  nn.Conv2d(in_channels = 512,out_channels = 512,kernel_size = 3, padding =1),
                  nn.BatchNorm2d(512),
                  nn.ReLU(),
                  nn.MaxPool2d(kernel_size=2, stride=2),
                  nn.Dropout2d(0.5) )

    self.fc =     nn.Sequential(
                  nn.Linear(512,100),
                  nn.Dropout(0.5),
                  nn.BatchNorm1d(100),
                  nn.ReLU(),
                  nn.Dropout(0.5),
                  nn.Linear(100,10), )
                  
  def forward(self,x):
    out = self.block1(x)
    out = self.block2(out)
    out = self.block3(out)
    out = self.block4(out)
    out = self.block5(out)
    out = out.view(out.size(0),-1)
    out = self.fc(out)

    return out


Model initialisation, Loss function and Optimizer

In [ ]:
model = VGG16()
if torch.cuda.is_available():
  model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.001,momentum = 0.9,weight_decay = 0.006)

model.eval()

VGG16(
  (block1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout2d(p=0.3, inplace=False)
  )
  (block2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7):

**Define CW Attack**

In [ ]:
# CW-L2 Attack
# Based on the paper, i.e. not exact same version of the code on https://github.com/carlini/nn_robust_attacks
# (1) Binary search method for c, (2) Optimization on tanh space, (3) Choosing method best l2 adversaries is NOT IN THIS CODE.
def cw_l2_attack(model, images, labels, targeted=False, c=1e-4, kappa=0, max_iter=1000, learning_rate=0.01) :
    
    images = images.to(device)     
    labels = labels.to(device)

    # Define f-function
    def f(x) :

        outputs = model(x)
        one_hot_labels = torch.eye(len(outputs[0]))[labels].to(device)
      
        i, _ = torch.max((1-one_hot_labels)*outputs, dim=1)
        j = torch.masked_select(outputs, one_hot_labels.byte())
        
        # If targeted, optimize for making the other class most likely 
        if targeted :
            return torch.clamp(i-j, min=-kappa)
        
        # If untargeted, optimize for making the other class most likely 
        else :
            return torch.clamp(j-i, min=-kappa)
    
    w = torch.zeros_like(images, requires_grad=True).to(device)
    
    optimizer = optim.Adam([w], lr=learning_rate)
    
    prev = 1e10
    
    for step in range(max_iter) :

        a = 1/2*(nn.Tanh()(w) + 1)

        loss1 = nn.MSELoss(reduction='sum')(a, images)
        loss2 = torch.sum(c*f(a))

        cost = loss1 + loss2

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        # Early Stop when loss does not converge.
        if step % (max_iter//10) == 0 :
            if cost > prev :
                print('Attack Stopped due to CONVERGENCE....')
                return a
            prev = cost
        
        print('- Learning Progress : %2.2f %%        ' %((step+1)/max_iter*100), end='\r')

    attack_images = 1/2*(nn.Tanh()(w) + 1)
    
    return attack_images

In [ ]:
def showAdversarialImages(img, outputFolderPath):
    npimg = img.numpy()

    #To make a figure without the frame :
    fig = plt.figure(frameon=False)
    fig.set_size_inches(0.456521,0.456521)
    #To make the content fill the whole figure
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)

    ax.imshow(np.transpose(npimg,(1,2,0)), aspect='auto')
   
    plt.xticks([])
    plt.yticks([])
    
    plt.gcf()
    
    fileName = str(len(os.listdir(outputFolderPath)) + 1) + '.png'

    plt.savefig(os.path.join(outputFolderPath, fileName))

Creating output main directory and class directories 

In [ ]:
import os
import cv2
output_base_dir = "/content/drive/MyDrive/mashfi/GTZAN_AdvarsarialAttack/output_12aug"
classes = ('blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock')
image_counts = [0, 0, 0, 0,0, 0, 0, 0, 0, 0]
if os.path.exists(output_base_dir):
  print("folder exist")
  
else:
  os.makedirs(output_base_dir)
  print("created folder : ", output_base_dir)
  

for i in range(len(classes)):
  label_path = os.path.join(output_base_dir, classes[i])
  if not os.path.exists(label_path):
    os.makedirs(label_path)


created folder :  /content/drive/MyDrive/mashfi/GTZAN_AdvarsarialAttack/output_12aug


In [ ]:
# print("Attack Image & Predicted Label")

model.eval()

correct = 0
total = 0

print(f"trainloader len {len(trainloader)}")
print(f" image length {len(images)}")
print(f"item length {len(images[0].cpu().data)}")
image_no = 0
train_no = 0
for images, labels in trainloader:
  print(f" labels {labels}Image no {image_no}")
  image_no += 1
  images = cw_l2_attack(model, images, labels, targeted=False, c=0.1)
  labels = labels.to(device)
  print(f"labels {labels}")
  outputs = model(images)
  
  _, pre = torch.max(outputs.data, 1)
  print(f"pre {pre}")

  total += 1
  correct += (pre == labels).sum()

  for index in range(len(images)):
    if pre[i] != labels[i] and image_counts[pre[i].item()]<101:
      class_index = pre[i].item()
      image_counts[class_index] +=1
      image_path = os.path.join(output_base_dir,classes[class_index]+'/' + str(image_counts[class_index]) + '.png')
      
      showAdversarialImages(images[i].cpu().data,os.path.join(output_base_dir,classes[class_index]))
